# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком.

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

# 1. Подготовка данных

In [93]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle

#Код от ревьюера
import warnings
warnings.filterwarnings('ignore')

In [94]:
# Посмотрим на данные
churn = pd.read_csv('./Churn.csv')
churn.info()
churn.head(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


# 2. Исследование задачи

In [95]:
# Преобразуем данные по недвижимости клиентов Tenure, предположим, что nan значения - недвижимость отсутствует,
# приведем к типу int
churn['Tenure'] = churn['Tenure'].fillna(0.0).astype('int')


In [96]:
# Посмотрим на распределение ушедших клиентов
churn.groupby('Exited')['Exited'].count()

Exited
0    7963
1    2037
Name: Exited, dtype: int64

Будем решать задачу классификации, т.к. целевой признак является категориальным.
Наблюдается дисбаланс классов, ушедших клиентов почти в 4 раза больше, чем оставшихся.
Обучим модель с дисбалансом, потом исправим это.

In [97]:
# Удалим столбцы, которые не влияют на обучение модели
useless_data = ['RowNumber', 'CustomerId', 'Surname']
churn = churn.drop(useless_data, axis=1)
churn['Geography'] = pd.get_dummies(churn['Geography'], drop_first=True)
churn['Gender'] = pd.get_dummies(churn['Gender'], drop_first=True)

In [98]:
def prepare_data(data):
    target = data['Exited']
    features = data.drop('Exited', axis=1)
    return target, features

Спрятанной тестовой выборки в задании нет, поэтому будем разбивать исходный набор данных на три части: обучающую, валидационную и тестовую. Размеры тестового и валидационного наборов сделаем равными.

In [99]:
# Функция разбивает датасет на  обучающую, валидационную и тестовую выборки (60/20/20)
def split_data(features, target):
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.4, random_state=12345
    )

    features_valid, features_test, target_valid, target_test = train_test_split(
        features_valid, target_valid, test_size=0.5, random_state=12345
    )

    # масштабируем количественные признаки
    scaler = StandardScaler()
    numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
    scaler.fit(features_train[numeric])

    features_train[numeric] = scaler.transform(features_train[numeric])
    features_valid[numeric] = scaler.transform(features_valid[numeric])
    features_test[numeric] = scaler.transform(features_test[numeric])

    return features_train, features_valid, features_test, target_train, target_valid, target_test

In [100]:
# функция обучает модель и проверяет её предсказания на валидационной/тестовой выборке
def train_and_predict(data, model, estimators = 0, max_depth = 0, samplefunc=None, test=False):
    target, features = prepare_data(data)
    features_train, features_valid, features_test, target_train, target_valid, target_test = split_data(features, target)

    if samplefunc:
        features_train, target_train = samplefunc(features_train, target_train)
        
        print(target_train.value_counts())

    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)

    if estimators != 0:
        print('Количество деревьев - ', estimators)

    if max_depth != 0:
        print('Максимальная глубина - ', max_depth)

    print(f'F1-мера на валидационной выборке: {f1_score(target_valid, predicted_valid)}')

    if test:
        predictions_test = model.predict(features_test)
        print(f'F1-мера на тестовой выборке: {f1_score(target_test, predicted_test)}')

    return model

In [101]:
# Логистическая регрессия
data_ohe = pd.get_dummies(churn, drop_first=True)
model = LogisticRegression(solver='liblinear', random_state=12345)
train_and_predict(data_ohe, model)

F1-мера на валидационной выборке: 0.3333333333333333


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=12345, solver='liblinear', tol=0.0001,
                   verbose=0, warm_start=False)

Результаты модели логистической регрессии оказались низкими, как и ожидалось. F1-мера = 0.31

In [102]:
# Применим технику порядкового кодирования для категориальных признаков для моделей случайного леса и дерева решений
encoder = OrdinalEncoder()
data_ordinal = pd.DataFrame(encoder.fit_transform(churn), columns=churn.columns)

In [103]:
# Дерево решений
for max_depth in range(2, 22, 2):
    model = DecisionTreeClassifier(max_depth=max_depth, random_state=12345)
    train_and_predict(data_ordinal, model, estimators=0, max_depth=max_depth)

Максимальная глубина -  2
F1-мера на валидационной выборке: 0.5217391304347825
Максимальная глубина -  4
F1-мера на валидационной выборке: 0.5528700906344411
Максимальная глубина -  6
F1-мера на валидационной выборке: 0.5658093797276854
Максимальная глубина -  8
F1-мера на валидационной выборке: 0.5384615384615384
Максимальная глубина -  10
F1-мера на валидационной выборке: 0.5381294964028777
Максимальная глубина -  12
F1-мера на валидационной выборке: 0.5079365079365079
Максимальная глубина -  14
F1-мера на валидационной выборке: 0.5109114249037227
Максимальная глубина -  16
F1-мера на валидационной выборке: 0.4987654320987654
Максимальная глубина -  18
F1-мера на валидационной выборке: 0.48727272727272725
Максимальная глубина -  20
F1-мера на валидационной выборке: 0.4794188861985472


📝 Пока лучшие результаты 0.53 получили у модели с глубиной равной 8

In [104]:
# Случайный лес
for estimators in range(10, 201, 10):
    model = RandomForestClassifier(n_estimators=estimators, max_depth=8, random_state=12345)
    train_and_predict(data_ordinal, model, estimators)

Количество деревьев -  10
F1-мера на валидационной выборке: 0.5647425897035881
Количество деревьев -  20
F1-мера на валидационной выборке: 0.5691823899371069
Количество деревьев -  30
F1-мера на валидационной выборке: 0.5660377358490566
Количество деревьев -  40
F1-мера на валидационной выборке: 0.5727699530516431
Количество деревьев -  50
F1-мера на валидационной выборке: 0.5754276827371695
Количество деревьев -  60
F1-мера на валидационной выборке: 0.5696400625978091
Количество деревьев -  70
F1-мера на валидационной выборке: 0.5709828393135726
Количество деревьев -  80
F1-мера на валидационной выборке: 0.575
Количество деревьев -  90
F1-мера на валидационной выборке: 0.5682888540031397
Количество деревьев -  100
F1-мера на валидационной выборке: 0.5732283464566928
Количество деревьев -  110
F1-мера на валидационной выборке: 0.5691823899371069
Количество деревьев -  120
F1-мера на валидационной выборке: 0.5632911392405064
Количество деревьев -  130
F1-мера на валидационной выборке: 0

Модель случайного леса предсказывает чуть лучше, чем дерево решений.
Лучший результат `0.54` достигается при количестве деревьев равном 10

## 3. Борьба с дисбалансом

Теперь сбалансируем классы и обучим модели с новым гиперпараметром `class_weight='balanced'`.

In [105]:
# Логистическая регрессия
model = LogisticRegression(class_weight='balanced', solver='liblinear', random_state=12345)
train_and_predict(data_ohe, model)

F1-мера на валидационной выборке: 0.4914383561643835


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=12345, solver='liblinear', tol=0.0001,
                   verbose=0, warm_start=False)

Значение F1-меры увеличилось до 0.48. Все равно такой результат не подходит.

In [106]:
# Дерево решений
for max_depth in range(2, 22, 2):
    model = DecisionTreeClassifier(class_weight='balanced', max_depth=max_depth, random_state=12345)
    train_and_predict(data_ordinal, model, estimators=0, max_depth=max_depth)

Максимальная глубина -  2
F1-мера на валидационной выборке: 0.541015625
Максимальная глубина -  4
F1-мера на валидационной выборке: 0.5277777777777778
Максимальная глубина -  6
F1-мера на валидационной выборке: 0.5581835383159887
Максимальная глубина -  8
F1-мера на валидационной выборке: 0.5385365853658537
Максимальная глубина -  10
F1-мера на валидационной выборке: 0.5160662122687439
Максимальная глубина -  12
F1-мера на валидационной выборке: 0.5025641025641027
Максимальная глубина -  14
F1-мера на валидационной выборке: 0.476510067114094
Максимальная глубина -  16
F1-мера на валидационной выборке: 0.47980997624703087
Максимальная глубина -  18
F1-мера на валидационной выборке: 0.46737841043890865
Максимальная глубина -  20
F1-мера на валидационной выборке: 0.4620938628158845


Значение F1-меры немного выросло: 0.54

In [107]:
# Случайный лес
for estimators in range(10, 201, 10):
    model = RandomForestClassifier(class_weight='balanced', n_estimators=estimators, max_depth=8, random_state=12345)
    train_and_predict(data_ordinal, model, estimators=estimators)

Количество деревьев -  10
F1-мера на валидационной выборке: 0.6263498920086392
Количество деревьев -  20
F1-мера на валидационной выборке: 0.6223698781838316
Количество деревьев -  30
F1-мера на валидационной выборке: 0.6172566371681416
Количество деревьев -  40
F1-мера на валидационной выборке: 0.6209944751381216
Количество деревьев -  50
F1-мера на валидационной выборке: 0.6226622662266227
Количество деревьев -  60
F1-мера на валидационной выборке: 0.6245919477693145
Количество деревьев -  70
F1-мера на валидационной выборке: 0.6281352235550708
Количество деревьев -  80
F1-мера на валидационной выборке: 0.6211453744493393
Количество деревьев -  90
F1-мера на валидационной выборке: 0.623059866962306
Количество деревьев -  100
F1-мера на валидационной выборке: 0.6189427312775331
Количество деревьев -  110
F1-мера на валидационной выборке: 0.6185792349726776
Количество деревьев -  120
F1-мера на валидационной выборке: 0.6175824175824175
Количество деревьев -  130
F1-мера на валидационно

Получили самое высокое значение среди всех моделей F1-меры: 0.61. Достигнуто с 100 деревьями.

### Увеличение выборки

Переобучим модель с балансировкой редких классов методом upsampling.

In [108]:
def upsample(features, target, repeat=4):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

#### Логистическая регрессия

In [109]:
model = LogisticRegression(solver='liblinear', random_state = 12345)
train_and_predict(data_ohe, model, samplefunc = upsample)

0    4804
1    4784
Name: Exited, dtype: int64
F1-мера на валидационной выборке: 0.4918594687232219


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=12345, solver='liblinear', tol=0.0001,
                   verbose=0, warm_start=False)

Значение F1-меры почти не изменилось.

#### Дерево решений

In [110]:
for max_depth in range(2, 22, 2):
    model = DecisionTreeClassifier(max_depth = max_depth, random_state = 12345)
    train_and_predict(data_ordinal, model, max_depth = max_depth, samplefunc = upsample)

0.0    4804
1.0    4784
Name: Exited, dtype: int64
Максимальная глубина -  2
F1-мера на валидационной выборке: 0.541015625
0.0    4804
1.0    4784
Name: Exited, dtype: int64
Максимальная глубина -  4
F1-мера на валидационной выборке: 0.5277777777777778
0.0    4804
1.0    4784
Name: Exited, dtype: int64
Максимальная глубина -  6
F1-мера на валидационной выборке: 0.5581835383159887
0.0    4804
1.0    4784
Name: Exited, dtype: int64
Максимальная глубина -  8
F1-мера на валидационной выборке: 0.538160469667319
0.0    4804
1.0    4784
Name: Exited, dtype: int64
Максимальная глубина -  10
F1-мера на валидационной выборке: 0.5213082259663032
0.0    4804
1.0    4784
Name: Exited, dtype: int64
Максимальная глубина -  12
F1-мера на валидационной выборке: 0.5145228215767635
0.0    4804
1.0    4784
Name: Exited, dtype: int64
Максимальная глубина -  14
F1-мера на валидационной выборке: 0.489795918367347
0.0    4804
1.0    4784
Name: Exited, dtype: int64
Максимальная глубина -  16
F1-мера на валидац

Здесь значение F1-меры так же почти не изменилось по сравнению с балансировкой.

#### Случайный лес

In [111]:
for estimators in range(10, 201, 10):
    model = RandomForestClassifier(n_estimators = estimators, max_depth=12, random_state = 12345)
    train_and_predict(data_ordinal, model, estimators = estimators, samplefunc = upsample)

0.0    4804
1.0    4784
Name: Exited, dtype: int64
Количество деревьев -  10
F1-мера на валидационной выборке: 0.5886363636363636
0.0    4804
1.0    4784
Name: Exited, dtype: int64
Количество деревьев -  20
F1-мера на валидационной выборке: 0.6096131301289567
0.0    4804
1.0    4784
Name: Exited, dtype: int64
Количество деревьев -  30
F1-мера на валидационной выборке: 0.6083916083916083
0.0    4804
1.0    4784
Name: Exited, dtype: int64
Количество деревьев -  40
F1-мера на валидационной выборке: 0.6107226107226107
0.0    4804
1.0    4784
Name: Exited, dtype: int64
Количество деревьев -  50
F1-мера на валидационной выборке: 0.61629279811098
0.0    4804
1.0    4784
Name: Exited, dtype: int64
Количество деревьев -  60
F1-мера на валидационной выборке: 0.627497062279671
0.0    4804
1.0    4784
Name: Exited, dtype: int64
Количество деревьев -  70
F1-мера на валидационной выборке: 0.6214953271028038
0.0    4804
1.0    4784
Name: Exited, dtype: int64
Количество деревьев -  80
F1-мера на валид

Для модели случайного леса метод upsampling выдал почти такой же результат как и при балансировке. Возможный выигрыш в сотых долях не такой существенный и можно использовать параметр class_weight на тестовой выборке.

### Уменьшение выборки

In [112]:
def downsample(features, target, fraction = 0.25):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones]
    )
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones]
    )
    
    return features_downsampled, target_downsampled

#### Логистическая регрессия

In [113]:
model = LogisticRegression(solver='liblinear', random_state=12345)
train_and_predict(data_ohe, model, samplefunc=downsample)

0    1201
1    1196
Name: Exited, dtype: int64
F1-мера на валидационной выборке: 0.4918594687232219


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=12345, solver='liblinear', tol=0.0001,
                   verbose=0, warm_start=False)

#### Дерево решений

In [114]:
for max_depth in range(2, 22, 2):
    model = DecisionTreeClassifier(max_depth=max_depth, random_state=12345)
    train_and_predict(data_ordinal, model, max_depth=max_depth, samplefunc=downsample)

0.0    1201
1.0    1196
Name: Exited, dtype: int64
Максимальная глубина -  2
F1-мера на валидационной выборке: 0.5394495412844036
0.0    1201
1.0    1196
Name: Exited, dtype: int64
Максимальная глубина -  4
F1-мера на валидационной выборке: 0.5357737104825291
0.0    1201
1.0    1196
Name: Exited, dtype: int64
Максимальная глубина -  6
F1-мера на валидационной выборке: 0.5722379603399433
0.0    1201
1.0    1196
Name: Exited, dtype: int64
Максимальная глубина -  8
F1-мера на валидационной выборке: 0.5297397769516728
0.0    1201
1.0    1196
Name: Exited, dtype: int64
Максимальная глубина -  10
F1-мера на валидационной выборке: 0.5279850746268657
0.0    1201
1.0    1196
Name: Exited, dtype: int64
Максимальная глубина -  12
F1-мера на валидационной выборке: 0.5048715677590787
0.0    1201
1.0    1196
Name: Exited, dtype: int64
Максимальная глубина -  14
F1-мера на валидационной выборке: 0.4802686817800168
0.0    1201
1.0    1196
Name: Exited, dtype: int64
Максимальная глубина -  16
F1-мера н

#### Случайный лес

In [115]:
for estimators in range(10, 201, 10):
    model = RandomForestClassifier(n_estimators=estimators, max_depth=2, random_state=12345)
    train_and_predict(data_ordinal, model, estimators=estimators, samplefunc=downsample)

0.0    1201
1.0    1196
Name: Exited, dtype: int64
Количество деревьев -  10
F1-мера на валидационной выборке: 0.5548617305976806
0.0    1201
1.0    1196
Name: Exited, dtype: int64
Количество деревьев -  20
F1-мера на валидационной выборке: 0.5818847209515096
0.0    1201
1.0    1196
Name: Exited, dtype: int64
Количество деревьев -  30
F1-мера на валидационной выборке: 0.5744493392070484
0.0    1201
1.0    1196
Name: Exited, dtype: int64
Количество деревьев -  40
F1-мера на валидационной выборке: 0.5693950177935942
0.0    1201
1.0    1196
Name: Exited, dtype: int64
Количество деревьев -  50
F1-мера на валидационной выборке: 0.5622280243690164
0.0    1201
1.0    1196
Name: Exited, dtype: int64
Количество деревьев -  60
F1-мера на валидационной выборке: 0.5626643295354952
0.0    1201
1.0    1196
Name: Exited, dtype: int64
Количество деревьев -  70
F1-мера на валидационной выборке: 0.5618556701030928
0.0    1201
1.0    1196
Name: Exited, dtype: int64
Количество деревьев -  80
F1-мера на ва

---

Значения F1-меры у моделей Логистической регрессии и Дерева решений почти такие же как и с балансировкой, а результаты для модели Случайного леса оказались даже хуже.

Соответственно, на тестовой выборке будем использовать балансировку через параметр class_weight.

## 4. Тестирование модели

Протестируем модель случайного леса с глубиной 8 и 100 деревьями + балансировкой классов на тестовой выборке:

In [116]:
target, features = prepare_data(data_ordinal)
features_train, features_valid, features_test, target_train, target_valid, target_test = split_data(features, target)

model = RandomForestClassifier(class_weight='balanced', n_estimators=100, max_depth=8, random_state=12345)
model.fit(features_train, target_train)

predicted_valid = model.predict(features_valid)
print('valid:', f1_score(target_valid, predicted_valid))

predicted_test = model.predict(features_test)
print('test:', f1_score(target_test, predicted_test))

valid: 0.6189427312775331
test: 0.6069868995633186


На валидационной выборке получили F1-меру равную 0.62, а на тестовой 0.6.
Посчитаем AUC-ROC для нашей модели и посмотрим насколько она лучше случайной модели:

In [117]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

roc_auc_score(target_valid, probabilities_one_valid)

0.8542287940285146

AUC-ROC нашей модели 0.84, что является вполне неплохим показателем и лучше случайной модели на 0.34

# Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: задача исследована
    - [x]  Исследован баланс классов
    - [x]  Изучены модели без учёта дисбаланса
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 3: учтён дисбаланс
    - [x]  Применено несколько способов борьбы с дисбалансом
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 4: проведено тестирование
- [x]  Удалось достичь *F1*-меры не менее 0.59
- [x]  Исследована метрика *AUC-ROC*